In [13]:
# parse dev data function
def ParseJson(data_json):
    paragraphs = data_json['data'][0]['paragraphs']
    idx_list = list()
    context_list = list()
    ques_list = list()
    ans_list = list()
    for para in paragraphs:
        context = para['context']
        idx = para['qas'][0]['id']
        ques = para['qas'][0]['question']
        ans = '|'.join([ans['text'] for ans in para['qas'][0]['answers']])
        idx_list.append(idx)
        context_list.append(context)
        ques_list.append(ques)
        ans_list.append(ans)
    return idx_list, context_list, ques_list, ans_list

In [14]:
# dev data
import json
dev_file = '/home/t-honli/data/squad_EQnA/dev-v1.1.json' 
dev_data = json.load(open(dev_file, "r"))

In [15]:
import pandas as pd
idx, context, ques, ans = ParseJson(dev_data)
dev_pd = pd.DataFrame({'id':idx, 'context':context, 'question':ques, 'ground_truth':ans}, columns=['id', 'context', 'question', 'ground_truth'])
dev_pd.head(5)

,id,context,question,ground_truth
0,9200ae8372eadcb43e5cfb816f185ef9,"Start Up & Rev of a 2006 Acura RL SH - AWD.This has a 3.5L V6 making a decent 290 HP while paired to a 5 Speed Auto.The interior in this was very classy , roomy , & Comfortable .",2006 acura rl horsepower,290 HP
1,b7d7518b27e56604c9b868ba25565222,"The income tax withholding rate remains at 4.25 % for tax year 2015 . However , the personal exemption amount for tax year 2015 will change to $4,000 . You may continue to use 2014 Michigan Income...",2015 michigan withholding tax rate,4.25 %
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon color expertise with this Vidal Sassoon Pro Series Hair Color in 5RR Medium Vibrant Red . It offers 100 percent gray coverage in a permanent color form...,5rr hair color,Red
3,92484e0e63e92f1412854e9516b8f8c3,"Effects depend on the substance : hydrogen peroxide removes a bleached layer of skin , while nitric acid causes a characteristic color change to yellow in the skin . Chemical burns may occur throu...",acid that burns through skin,nitric acid|hydrogen peroxide
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who plays Jaime Lannister in HBO 's popular television series Game Of Thrones , has been spotted in Hobart , where he has spent time taking in the city 's c...",actor who plays jamie lannister,Nikolaj Coster|Waldau


In [16]:
# bidaf ans
import json
import os
file = '/home/t-honli/bi-att-flow/out/basic/00_EQnA/answer/test-020000_multi.json'
bidaf = json.load(open(file, "r"))

In [17]:
dev_pd['bidaf'] = dev_pd.apply(lambda row: bidaf[row['id']], axis=1)

In [18]:
# prod model
import pandas as pd
col_names = ['id', 'Query', 'Url', 'Answer', 'AnswerTokenList', 'ParaseSpan', 'phrase', 'Label', 'Probability']
prod = pd.read_csv('/home/t-honli/data/squad_EQnA/EQnA_Highlighting_Test_ProdModel.tsv', header=None, sep='\t', names=col_names, dtype=str).fillna('')

print (len(prod))

# 根据answer，query，url生成hash_id
import hashlib
def GetHashCode(context):
    hash = hashlib.md5()
    hash.update(context.encode('utf-8'))
    return hash.hexdigest()
prod['hash_id'] = prod.apply(lambda row: GetHashCode(row['Answer'] + ' ' + row['Query'] + ' ' + row['Url']), axis=1)

prod = prod[prod.apply(lambda row: len(row['phrase'])<32, axis=1)]
print (len(prod))
prod.head(5)

15560
15454


,id,Query,Url,Answer,AnswerTokenList,ParaseSpan,phrase,Label,Probability,hash_id
0,00021900a221a0b2e5c9a923c4f13813,where does the water from mississippi's rivers drain,http://en.wikipedia.org/wiki/Mississippi_River,"In the United States, the Mississippi River drains the majority of the area between crest of the Rocky Mountains and the crest of the Appalachian Mountains, except for various regions drained to H...","[""In"",""the"",""United"",""States"","","",""the"",""Mississippi"",""River"",""drains"",""the"",""majority"",""of"",""the"",""area"",""between"",""crest"",""of"",""the"",""Rocky"",""Mountains"",""and"",""the"",""crest"",""of"",""the"",""Appalachi...",79:79,rivers,0,0.01860236,9b6bef3c4ce179302bbad6ddf9b98832
1,0004dd703e44213707c923e548627774,good foods for pregnant women,http://www.netdoctor.co.uk/health_advice/facts/pregnantdiet.htm,"During pregnancy, a woman's body needs more iron than usual to produce all the blood needed to supply nutrition to the placenta. Good sources of iron are green vegetables such as broccoli and spin...","[""During"",""pregnancy"","","",""a"",""woman"",""'s"",""body"",""needs"",""more"",""iron"",""than"",""usual"",""to"",""produce"",""all"",""the"",""blood"",""needed"",""to"",""supply"",""nutrition"",""to"",""the"",""placenta"",""."",""Good"",""sourc...",4:4,woman,0,0.1246661,ac6273ec9adf9778fb33f09501f43874
2,00052a47dccf1667995ebffb3019b565,what continent is belize on,http://www.belize.com/where-is-belize,Belize is located on the Caribbean coast of northern Central America at 17°15′ north of the equator and 88°45′ west of the Prime Meridian on the Yucatán Peninsula. Central America is the isthmus t...,"[""Belize"",""is"",""located"",""on"",""the"",""Caribbean"",""coast"",""of"",""northern"",""Central"",""America"",""at"",""17°15"",""′"",""north"",""of"",""the"",""equator"",""and"",""88°45"",""′"",""west"",""of"",""the"",""Prime"",""Meridian"",""on...",19:19,88°45,0,0.02859148,2110154eb61df6e89f85c33c204492e0
3,00099640760fac2092756da3714c60a9,when was tom seaver born,http://www.baseball-almanac.com/players/player.php?p=seaveto01,"Tom Seaver was born on Friday, November 17, 1944, in Fresno, California. Seaver was 22 years old when he broke into the big leagues on April 13, 1967, with the New York Mets.","[""Tom"",""Seaver"",""was"",""born"",""on"",""Friday"","","",""November"",""17"","","",""1944"","","",""in"",""Fresno"","","",""California"",""."",""Seaver"",""was"",""22"",""years"",""old"",""when"",""he"",""broke"",""into"",""the"",""big"",""leagues"",...",17:17,Seaver,0,0.001268676,a272a8fdd6b0e63d75e39f732639c31f
4,000e8ade636d16933ddc53486a59a976,disease caused by deer ticks,http://www.niaid.nih.gov/topics/tickborne/Pages/Default.aspx,"Ticks transmit ehrlichiosis and anaplasmosis, both bacterial diseases. Babesiosis is caused by parasites carried by deer ticks. These diseases are found in several states.","[""Ticks"",""transmit"",""ehrlichiosis"",""and"",""anaplasmosis"","","",""both"",""bacterial"",""diseases"",""."",""Babesiosis"",""is"",""caused"",""by"",""parasites"",""carried"",""by"",""deer"",""ticks"",""."",""These"",""diseases"",""are""...",0:0,Ticks,0,0.008481221,54ee3a3b1b57174ee8ee50625d20ee74


In [20]:
# 获取hash_id, [(phrase,score)]
multi_phrase_dict = {}
def getPhrase(row):
    if row['hash_id'] not in multi_phrase_dict:
        multi_phrase_dict[row['hash_id']] = list()
    row['Probability'] = '%.4f' % float(row['Probability'])
    multi_phrase_dict[row['hash_id']].append(row['phrase'] + ":" + row['Probability'])
prod.apply(getPhrase, axis=1)
print (len(prod))

15454


In [21]:
# sort
for idx in list(multi_phrase_dict.keys()):
    multi_phrase_dict[idx].sort(key=lambda x: float(x.split(':')[-1]), reverse=True)

In [22]:
print (list(multi_phrase_dict.items())[:5])

[('9b6bef3c4ce179302bbad6ddf9b98832', ['Alabama:0.8560', 'Rio Grande:0.7439', 'Saint Lawrence River:0.7311', 'Tombigbee:0.6698', 'Chattahoochee:0.5844', 'the Atlantic Ocean:0.4905', 'Red River of the North:0.4180', 'the United States:0.3123', 'the Great Lakes:0.3058', 'waterways:0.2184', 'the Mississippi:0.0737', 'drained:0.0447', 'coastal:0.0258', 'rivers:0.0244', 'rivers:0.0186', 'regions:0.0173']), ('ac6273ec9adf9778fb33f09501f43874', ['spinach:0.9555', 'broccoli:0.9459', 'green vegetables:0.9092', 'wholemeal bread:0.8101', 'blood:0.6137', 'produce:0.5852', 'iron:0.4314', 'placenta:0.2961', 'iron:0.2657', 'nutrition:0.2498', 'pregnancy:0.1735', 'woman:0.1247', 'body:0.0602']), ('2110154eb61df6e89f85c33c204492e0', ['South America:0.9047', 'North America:0.6379', 'Yucatán Peninsula:0.1944', 'the Caribbean:0.1486', 'Prime Meridian:0.1198', 'the equator:0.0432', '17°15:0.0291', '88°45:0.0286', 'north:0.0249', 'west:0.0137', 'Belize:0.0076']), ('a272a8fdd6b0e63d75e39f732639c31f', ['Novem

In [23]:
dev_pd['prod'] = dev_pd.apply(lambda row: '|'.join(multi_phrase_dict[row['id']]), axis=1)

In [24]:
pd.set_option('max_colwidth', 200)
dev_pd.head(5)

,id,context,question,ground_truth,bidaf,prod
0,9200ae8372eadcb43e5cfb816f185ef9,"Start Up & Rev of a 2006 Acura RL SH - AWD.This has a 3.5L V6 making a decent 290 HP while paired to a 5 Speed Auto.The interior in this was very classy , roomy , & Comfortable .",2006 acura rl horsepower,290 HP,290 HP|3.5L|290 HP while paired to a 5,290 HP:0.8149|V6:0.4651|AWD.This:0.3202|3.5L:0.1977|5:0.1863|has a:0.0248|2006:0.0102|Acura RL:0.0096
1,b7d7518b27e56604c9b868ba25565222,"The income tax withholding rate remains at 4.25 % for tax year 2015 . However , the personal exemption amount for tax year 2015 will change to $4,000 . You may continue to use 2014 Michigan Income...",2015 michigan withholding tax rate,4.25 %,"4.25 %|4.25 % for tax year 2015 . However , the personal exemption amount for tax year 2015 will change to $4,000|4.25","4.25 %:0.8581|$4,000:0.6614|2014:0.0384|personal exemption:0.0171|Withholding:0.0084|withholding:0.0077|Income Tax:0.0064|income tax:0.0062|2015:0.0059|2015:0.0055|Michigan:0.0052|may:0.0042"
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon color expertise with this Vidal Sassoon Pro Series Hair Color in 5RR Medium Vibrant Red . It offers 100 percent gray coverage in a permanent color form...,5rr hair color,Red,gray|100 percent|Red,Red:0.9268|gray:0.9099|Vidal Sassoon:0.0242|100 percent:0.0170|Vidal Sassoon:0.0156|color:0.0060|color:0.0047|Hair Color:0.0040|5RR:0.0034
3,92484e0e63e92f1412854e9516b8f8c3,"Effects depend on the substance : hydrogen peroxide removes a bleached layer of skin , while nitric acid causes a characteristic color change to yellow in the skin . Chemical burns may occur throu...",acid that burns through skin,nitric acid|hydrogen peroxide,"hydrogen peroxide|hydrogen peroxide removes a bleached layer of skin , while nitric acid|hydrogen",hydrogen peroxide:0.4230|nitric acid:0.2648|yellow:0.0626|Chemical burns:0.0581|color:0.0269|body:0.0147|may:0.0079|skin:0.0059|skin:0.0057|skin:0.0056
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who plays Jaime Lannister in HBO 's popular television series Game Of Thrones , has been spotted in Hobart , where he has spent time taking in the city 's c...",actor who plays jamie lannister,Nikolaj Coster|Waldau,Nikolaj Coster|Nikolaj Coster - Waldau|Nikolaj,Jaime:0.7372|Nikolaj Coster:0.7275|Waldau:0.0507|Hobart:0.0208|HBO:0.0192|Game Of Thrones:0.0083|television series:0.0057|Actor:0.0035|has been:0.0034


In [25]:
dev_pd.to_csv('./sample_multi.tsv', header=True, index=None, sep='\t')

In [ ]:
# SampleAndCompactMulti.ipynb add threshold
# generate_prodformat_label.ipynb change bidaf data to prod format